In [4]:
import numpy as np

In [5]:
v = np.array([-2,0])

T = np.identity(3)
T[0:2,2] = v
print("Translate matrix:\n", T)

Translate matrix:
 [[ 1.  0. -2.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


In [12]:
def Trans(v):
    T = np.identity(3)
    T[0:2,2] = v
    return T

def Rot(α):
    return np.array([[np.cos(α), -np.sin(α), 0], [np.sin(α), np.cos(α), 0], [0,0,1]])

Sy = np.array([[-1, 0, 0],[0,1,0],[0,0,1]])

Rot(np.pi/4)

array([[ 0.70710678, -0.70710678,  0.        ],
       [ 0.70710678,  0.70710678,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [8]:
def symmetry(point):
    point3d = np.array([point[0], point[1],1])
    res = T @ point3d
    res = Rot(np.pi/4) @ res
    res[0] = -res[0]
    res = Rot(-np.pi/4) @ res
    res += np.array([2,0,1])
    return res[:2]
    

In [9]:
symmetry([2,0])

array([2., 0.])

In [10]:
symmetry([0,0])

array([ 2., -2.])

In [16]:
my_mat_sym =  Trans([2,0]) @ Rot(-np.pi / 4) @ Sy @ Rot(np.pi/4) @ Trans([-2,0])

def symmetry2(point):
    point3d = np.array([point[0], point[1],1])
    res = my_mat_sym @ point3d
    return res[:2]


In [17]:
my_mat_sym

array([[-1.99673462e-16,  1.00000000e+00,  2.00000000e+00],
       [ 1.00000000e+00,  1.79380389e-16, -2.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [92]:
symmetry2([0,0])

array([ 2., -2.])

In [19]:
def Rot(α):
    return np.array([[np.cos(α), -np.sin(α)], [np.sin(α), np.cos(α)]])

def angle_terre(t):
    return t * 2 * np.pi / 365

def angle_lune(t):
    return t * 2 * np.pi * (1/365 + 1/29)

def moon_position(t):
    terre = np.arrtay([150550, 0])
    lune = np.array([384, 0])
    
    terre = Rot(angle_terre(t))
    lune = Rot(angle_lune(t))
    
    return terre + lune

In [20]:
np.correlate?

In [21]:
data = [[2.58801783, 3.13826759, 4.72942079, 3.01136245, 4.24651163,  3.34635203, 1.33927513, 3.8661015 , 3.70501302, 2.97225954],
        [3.11911504, 4.45671258, 4.65381438, 3.90961598, 4.99041127, 4.68080288, 2.40431166, 1.59106122, 2.22220669, 2.71333667],
        [0.26496776, 0.03572671, 0.3118583 , 0.20960038, 0.07581288, 0.16855627, 0.12381024, 0.12224255, 0.21919762, 0.38974431],

        [5.08512469, 4.34647593, 7.66508604, 6.51302853, 7.26870321, 7.79430609, 3.95380041, 2.66393951, 3.74730669, 4.44003053],

        [1.93135633, 1.46295984, 3.22940609, 2.03313999, 2.89821614, 0.40726548, 0.22551519, 2.61514052, 2.50802962, 2.22258118]]


In [23]:
for i in range(5):
    for j in range(5):
        print(i,j, np.correlate(data[i], data[j]))

0 0 [116.51506444]
0 1 [118.36653744]
0 2 [6.39891136]
0 3 [182.28945973]
0 4 [70.96624409]
1 0 [118.36653744]
1 1 [133.96096012]
1 2 [6.46057609]
1 3 [203.24408665]
1 4 [68.1986158]
2 0 [6.39891136]
2 1 [6.46057609]
2 2 [0.47705148]
2 3 [10.49011891]
2 4 [4.0492422]
3 0 [182.28945973]
3 1 [203.24408665]
3 2 [10.49011891]
3 3 [315.99397447]
3 4 [105.54100923]
4 0 [70.96624409]
4 1 [68.1986158]
4 2 [4.0492422]
4 3 [105.54100923]
4 4 [47.11852936]


In [88]:

def solve_gauss(A, b):
    A = A.astype(np.float64)   # si A a des entiers, on va avoir des erreurs de calculs
    b = b.astype(np.float64)   # si A a des entiers, on va avoir des erreurs de calculs
    permutation = np.arange(len(A))
    for i in range(len(A)-1):
        pos_pivot = np.argmax(np.abs(A[i:, i:]))
        pos_line = i + pos_pivot // (len(A) - i)
        pos_column = i + pos_pivot % (len(A) - i)
        # echange des lignes
        A[[i, pos_line]] = A[[pos_line, i]]
        b[[i, pos_line]] = b[[pos_line, i]]
        # echange colonne
        A[:, [i, pos_column]] = A[:, [pos_column, i]]
        permutation[[i, pos_column]] = permutation[[pos_column, i]]
        coefs = - A[i+1:,i] / A[i,i]  # Normalement il faut tester que A[i,i] != 0
        A[i+1:, i:] += np.outer(coefs, A[i, i:]) # ou np.einsum('i,j -> ij', coefs, A[i, i:])
        b[i+1:] += coefs * b[i]                  # multiplication terme à terme
    # A est maintenant triangulaire surpérieur
    res = np.zeros(len(b), dtype=b.dtype)
    res[-1] = b[-1] / A[-1,-1]
    for i in range(len(A)-1)[::-1]:
        res[i] = (b[i] - A[i,i+1:] @ res[i+1:]) / A[i,i]
    return res[permutation.argsort()]


In [89]:
A = np.random.randint(10, size=(5,5)).astype('float64')
#b = A.sum(axis=1)
b = np.ones(5)
A,b

(array([[0., 9., 2., 3., 9.],
        [3., 7., 5., 7., 5.],
        [2., 0., 7., 3., 2.],
        [5., 0., 7., 2., 6.],
        [4., 0., 6., 5., 0.]]),
 array([1., 1., 1., 1., 1.]))

In [90]:
res = solve_gauss(A,b)
res

1 [1 0 2 3 4]
2 [1 2 0 3 4]
4 [1 2 4 3 0]
4 [1 2 4 0 3]
[ 6.48214286  3.39285714 -4.76785714  4.         -7.07142857] [1 2 4 0 3]


array([ 4.        ,  6.48214286,  3.39285714, -7.07142857, -4.76785714])

In [91]:
A @ res

array([1., 1., 1., 1., 1.])